In [1]:
import cred
from datetime import datetime
import requests
from langchain.tools import Tool
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory



In [ ]:
# Step 1: Define the function
def get_train_details(train_number: str) -> str:
    """
    Query train information for a particular train number.
    
    Args:
        train_number (str): The number of the train to get the status for.
        
    Returns:
        str: train details for the train number.
    """
    # Use a free railway api (replace with your API key)
    api_key = cred.railway_token  # Replace with your API key
    # enter current date in dd-mm-yyyy format
    current_date = datetime.today().date().strftime("%d%m%Y")

    # complete_url variable to
    # store complete url address
    url = "https://irctc1.p.rapidapi.com/api/v1/getTrainScheduleV2"

    querystring = {"trainNo":f"{train_number}"}

    headers = {
        "x-rapidapi-key": "a14450995dmsh0e8047dace38a23p1e2e4bjsnc01d4f66ed0c",
        "x-rapidapi-host": "irctc1.p.rapidapi.com"
    }
    result = requests.get(url,headers=headers,params=querystring)
    if result.status_code == 200:
        response=result.json() 
        train_name = response["data"]["train_name"]
        source_station = response["data"]["origin"]["station_name"]
        destination_station = response["data"]["destination"]["station_name"]
        
        print(" train name : " + str(train_name)
            + "\n source station : " + str(source_station)
            + "\n destination station : "+ str(destination_station))
        return f"the train number {train_name} runs from station {source_station} to destinsation station {destination_station} ."
    else:
        return f"Failed to get data for {train_number}. Error: {result.json().get('message', 'Unknown error')}"

# Step 2: Wrap the function in a LangChain Tool
train_detail_tool = Tool(
    name="trainDetailsQuery",
    func=get_train_details,
    description="Get the details of train for train number."
)

# Step 3: Test the tool
# train_number = "22222"
# result = weather_tool.func(train_number)
# print(result)  # Prints the weather details for New York

# Step 4: (Optional) Add the tool to an agent
# from langchain.agents import initialize_agent
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# tools = [train_detail_tool]  # Add more tools if needed
# llm = HuggingFaceEndpoint(
#     repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
#     task="text-generation",
#     max_new_tokens=512,
#     do_sample=False,
#     repetition_penalty=1.03,
# )
# agent = initialize_agent(tools,llm=llm ,agent="zero-shot-react-description", verbose=True)

# # Use the agent
# response = agent.run("What is the name of train 22222")
# print(response)  

e:\Projects\GEN_AI\assist_ai_with_langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\dell nwc\AppData\Local\Temp\ipykernel_3052\2316323586.py:67: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initial

In [5]:
# Step 1: Define the function
def get_train_status(train_number: str) -> str:
    """
    Query current status of a train for a particular train number.
    
    Args:
        
        train_number (str): The number of the train to get the status for.
        
    Returns:
        str: train status for the train number.
    """
    
    current_date = datetime.today().date().strftime("%d%m%Y")

    # complete_url variable to
    # store complete url address
    url = "https://irctc1.p.rapidapi.com/api/v1/liveTrainStatus"

    querystring = {"trainNo":"22222","startDay":"1"}


    headers = {
        "x-rapidapi-key": "a14450995dmsh0e8047dace38a23p1e2e4bjsnc01d4f66ed0c",
        "x-rapidapi-host": "irctc1.p.rapidapi.com"
    }
    result = requests.get(url,headers=headers,params=querystring)
    llm = HuggingFaceEndpoint(
        repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
        task="text-generation",
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    )
    if result.status_code == 200:
        response=result.json() 
        chat_model = ChatHuggingFace(llm=llm)
        messages = [SystemMessage(content=f'You are a great assistant which help me with the query related to train curret status, for your refference here is the data for the train {result.content}'),
                    HumanMessage(content="give me the current status of train")]
        output=chat_model.invoke(messages)
        
        return output
    else:
        return f"Failed to get data for {train_number}. Error: {result.json().get('message', 'Unknown error')}"

# Step 2: Wrap the function in a LangChain Tool
train_status_tool = Tool(
    name="trainStatusQuery",
    func=get_train_status,
    description="Get the current status of train for train number."
)



# #Step 3: Test the tool
# train_number = "22222"
# result = train_status_tool.func(train_number)
# print(result)  # Prints the weather details for New York

# # Step 4: (Optional) Add the tool to an agent
# from langchain.agents import initialize_agent
# from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# tools = [train_detail_tool,train_status_tool]  # Add more tools if needed
# llm = HuggingFaceEndpoint(
#     repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
#     task="text-generation",
#     max_new_tokens=512,
#     do_sample=False,
#     repetition_penalty=1.03,
# )
# agent = initialize_agent(tools,llm=llm ,agent="zero-shot-react-description", verbose=True)

# # Use the agent
# response = agent.run("What is the status of train 22222")
# print(response)  

In [8]:
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

In [11]:
# Step 4: (Optional) Add the tool to an agent
tools = [train_detail_tool,train_status_tool,chat_model]  # Add more tools if needed
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)
memory = ConversationBufferMemory(memory_key="chat_history")
# agent = initialize_agent(tools,llm=llm ,agent="zero-shot-react-description", verbose=True)
# # Use the agent
# response = agent.run("What is the status of train 22222")
# print(response)  

C:\Users\dell nwc\AppData\Local\Temp\ipykernel_3052\2718818631.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [13]:
agent = initialize_agent(
    tools=[train_status_tool,train_detail_tool],  # Add multiple tools if needed
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,  # Ensures chat + tool usage
    memory=memory,  # Enables remembering past messages
    verbose=True  # Useful for debugging
)

In [14]:
while True:
    user_input = input("You: ")  # Take user input

    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break

    response = agent.run(user_input)  # Pass user input to the agent
    print(f"Chatbot: {response}")



> Entering new AgentExecutor chain...


Thought: Do I need to use a tool? No
AI: Hello! How can I assist you today? If you have any questions or need information about a specific topic, feel free to let me know.

> Finished chain.
Chatbot: Hello! How can I assist you today? If you have any questions or need information about a specific topic, feel free to let me know.


> Entering new AgentExecutor chain...
AI: Thought: Do I need to use a tool? No
AI: Hello! How can I assist you today? If you have any questions or need information about a specific topic, feel free to let me know.

> Finished chain.
Chatbot: Hello! How can I assist you today? If you have any questions or need information about a specific topic, feel free to let me know.
Chatbot: Goodbye!
